### Preprocess tiles

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Background

# Some styles are ignored
images = [Image.open(f"original/BOOM Graphics/BGPattern 0{i+2}.png").convert("RGBA") for i in range(8)]
image = Image.fromarray(np.vstack(images))
image.save("image/background.png")

plt.imshow(image)
plt.show()

In [ ]:
# Border

images = [Image.open(f"original/BOOM Graphics/Border 0{i+1}.png").convert("RGBA") for i in range(8)]
image = Image.fromarray(np.vstack(images))
image.save("image/border.png")

plt.imshow(image)
plt.show()

In [ ]:
# Breakable

image = Image.open(f"original/BOOM Graphics/Breakable Blocks.png").convert("RGBA")
image.save("image/breakable_wall.png")

plt.imshow(image)
plt.show()

In [ ]:
# Solid

image = Image.open(f"original/BOOM Graphics/Fixed Blocks.png").convert("RGBA")
image = Image.fromarray(np.vstack(np.split(np.array(image), 8, axis = 1)))
image.save("image/solid_wall.png")

plt.imshow(image)
plt.show()

In [ ]:
# Player

void = np.array([[[255, 255, 255, 0]] * 32] * 32, dtype=np.uint8)

for i in range(2):
    image = Image.open(f"original/BOOM Graphics/Player{i+1}.png").convert("RGBA")
    line = np.hstack(np.split(np.array(image), 2) + [void] * 2)
    image = Image.fromarray(np.vstack(np.split(line, 5, axis = 1)))
    image.save(f"image/player{i+1}.png")
    
    plt.imshow(image)
    plt.show()

In [ ]:
# Bomb

image = Image.open("original/BOOM Graphics/Bomb.png")
image.save("image/bomb.png")

plt.imshow(image)
plt.show()

In [ ]:
# Laser

image = Image.open("original/BOOM Graphics/Explosion.png")
image.save("image/laser.png")

plt.imshow(image)
plt.show()

In [ ]:
# Enemy

void = np.array([[[255, 255, 255, 0]] * 32] * 32, dtype=np.uint8)

for name in ["Soldier", "Sarge", "Lizzy", "Taur", "Gunner", "Thing", "Ghost", "Smoulder", "Skully", "Giggler"]:
    image = Image.open(f"original/BOOM Graphics/{name}.png").convert("RGBA")
    line = np.hstack(np.split(np.array(image), 2))
    n = line.shape[1] // 32
    if n%4:
        line = np.hstack([line] + (4 - (n%4)) * [void])

    image = Image.fromarray(np.vstack(np.split(line, 6, axis = 1)))
    image.save(f"image/{name.lower()}.png")
    
    plt.imshow(image)
    plt.show()

In [ ]:
# Panel

image = Image.open("original/BOOM Graphics/Panel.png")
image.save("image/panel.png")
print(image.size, " -> ", image.size[0] // 32, ",", image.size[1] // 32)

plt.imshow(image)
plt.show()

In [ ]:
# Player head

for i in range(2):
    image = Image.open(f"original/BOOM Graphics/Player {i+1} Head.png").convert("RGBA")
    image.save(f"image/player_head{i+1}.png")
    print(image.size)
    
    plt.imshow(image)
    plt.show()

In [ ]:
# Heart

image = Image.open(f"original/BOOM Graphics/Heart Icons.png").convert("RGBA")
np_image = np.array(image)[0:16]
image = Image.fromarray(np.hstack((np_image[:, 0:16], np_image[:, 18:34], np_image[:, 36:52])))  # Delete weird extra space
image.save("image/heart.png")

plt.imshow(image)
plt.show()

In [ ]:
# Shield

image = Image.open(f"original/BOOM Graphics/Shield.png").convert("RGBA")
image.save(f"image/shield.png")

plt.imshow(image)
plt.show()

###  Levels and maze

In [ ]:
import json
import xml.etree.ElementTree as ET

import bomberman

In [ ]:
players = {
    "X": 1,
    "Y": 2,
}

klasses = {
    "1": bomberman.model.entity.SolidWall,
    "2": bomberman.model.entity.BreakableWall,
    "A": bomberman.model.entity.Soldier,
    "B": bomberman.model.entity.Sarge,
    "C": bomberman.model.entity.Lizzy,
    "D": bomberman.model.entity.Taur,
    "E": bomberman.model.entity.Gunner,
    "F": bomberman.model.entity.Thing,
    "G": bomberman.model.entity.Ghost,
    "H": bomberman.model.entity.Smoulder,
    "I": bomberman.model.entity.Skully,
    "J": bomberman.model.entity.Giggler,
}

undecoded = set()


def parse_repr(maze_repr):
    """from original to actual"""
    maze = bomberman.model.maze.Maze((13, 15))
    for n, c in enumerate(maze_repr):
        if c == "0":
            continue
        i = n // 15
        j = n % 15
        if c in players:
            maze.player_spawns[players[c]] = bomberman.model.vector.Vector((float(i), float(j)))
            continue

        klass = klasses.get(c)
        if not klass:
            print(f"Undecoded '{c}' at ({i}, {j})")
            undecoded.add(c)
            continue

        maze.add_entity(klass(maze, bomberman.model.vector.Vector((float(i), float(j)))))

    return maze

In [ ]:
def parse_xml(level_xml):
    level = {}

    key = None
    for child in level_xml:
        if child.tag == "key":
            assert key is None
            key = child.text
        else:
            value = child.text
            if child.tag == "integer":
                value = int(value)
            assert key is not None
            level[key] = value
            key = None

    return level

In [ ]:
tree = ET.parse("original/BOOM Levels.xml")
root = tree.getroot()
levels = root.find("dict").find("array")

game = []

for i, level_xml in enumerate(levels):
    print("======================")
    print(i)
    level = parse_xml(level_xml)
    meta_data = {
        "style": i // 10,
        "maze_id": i + 1,
        "time": level["Time"],
    }

    maze = parse_repr(level["GridDescString"])
    print(maze)
    maze.save(f"maze/{meta_data['maze_id']}.txt")
    game.append(meta_data)
    
print("Undecoded chars:")
print(undecoded)
with open("game/boom.json", "w") as f:
    json.dump(game, f, indent=True)